# Prepare Syn Data

In [ ]:
import pandas as pd

data = {
    'darija_masked': [
        "أنا <mask> الكتاب.", "هو <mask> ف الدار.", "بغيت <mask> الماء.", "ال<mask> زوين بزاف.",
        "<mask> سميتك؟", "مشيت <mask> للسوق.", "غادي <mask> للمدرسة.", "كنت <mask> مع صحابي.",
        "كتبت رسالة <mask> الصديق ديالي.", "ما <mask> هاد الفيلم حيت ما عجبنيش.",
        "الراجل لي <mask> معاك هو خويا.", "فاش <mask> فالدار، كنقرا الكتوب.",
        "ما عرفتش <mask> غادي ندير.", "<mask> الحال اليوم؟", "ضربني <mask>.",
        "هاد الماكلة <mask> بزاف!", "كاين <mask> ديال الناس ف السوق.", "شفت <mask> ف الطريق.",
        "عطيني <mask>.", "عندي <mask> خوت.", "شريت <mask> كيلو ديال التفاح.",
        "البنت <mask> بزاف.", "واش نتا <mask>؟", "فين <mask> البارح؟", "ما عنديش <mask>.",
        "هذا <mask> ديالك؟", "<mask> كليتي؟", "غادي نمشي <mask> غدا.", "كنت <mask> البارح.",
        "عفاك، <mask> شوية.", "ما فهمتش <mask> قلتي.", "شحال <mask> الساعة؟", "بغيت <mask> قهوة.",
        "الدار <mask> نقية.", "<mask> خدام؟", "اليوم <mask> عيد ميلادي.", "كنحس <mask> عيان.",
        "التليفون ديالي <mask>.", "فين <mask> تسكن؟", "واش <mask> مزيان؟", "<mask> خاصك؟",
        "هادشي <mask> صعيب.", "أنا <mask> نتعلم الدارجة.", "الطقس <mask> زوين اليوم.",
        "عندي <mask> سؤال.", "ما كنعرفش <mask> ندير.", "بغيت <mask> نسافر.", "<mask> عام عندك؟",
        "أنا <mask> من المغرب.", "هاد الكتاب <mask> بزاف.", "<mask> تدير دابا؟"

    ],
    'target': [
        "قريت", "كاين / راه", "نشرب / شوية", "كتاب / فيلم / ماكلة", "شنو",
        "البارح / اليوم", "نمشي / نكون", "كنلعب / كندوي / كنهضر", "ل", "شفتش",
        "كان / كيهضر", "كنكون", "شنو", "كيداير", "البرد", "بنينة", "بزاف",
        "طوموبيل / راجل / كلب / شجرة", "الخبز / الماء / فلوس / ستيلو",
        "ثلاثة / ربعة / واحد", "واحد / جوج", "زوينة / صغيرة / كبيرة", "مكلخ / مغربي / هنا",
        "كنتي / مشيتي", "فلوس / الوقت", "الكتاب / القلم", "شنو / واش", "نسافر / نخدم",
        "مريض / خدام", "عاوني / عطيني", "شنو / داكشي", "ف / دابا", "نشرب / ناكل", "ديالي / عامرة",
        "فين / واش", "هو / ديال", "شويا / بزاف", "طافي / خسر", "كتسكن / ساكن", "كلشي / نتا",
        "شنو / واش", "بزاف / شوية", "كنحاول / باغي", "ديال / اليوم", "ليك / واحد",
        "هادشي / كيفاش", "نمشي / نجي", "شحال / من", "جاي / من", "زوين / غالي","أش / شنو"
    ],
    'darija_unmasked': [
        ['أنا قريت الكتاب.'],
        ['هو كاين ف الدار.', 'هو راه ف الدار.'],
        ['بغيت نشرب الماء.', 'بغيت شوية الماء.'],
        ['الكتاب زوين بزاف.', 'الفيلم زوين بزاف.', 'الماكلة زوين بزاف.'],
        ['شنو سميتك؟'],
        ['مشيت البارح للسوق.', 'مشيت اليوم للسوق.'],
        ['غادي نمشي للمدرسة.', 'غادي نكون للمدرسة.'],
        ['كنت كنلعب مع صحابي.', 'كنت كندوي مع صحابي.', 'كنت كنهضر مع صحابي.'],
        ['كتبت رسالة ل الصديق ديالي.'],
        ['ما شفتش هاد الفيلم حيت ما عجبنيش.'],
        ['الراجل لي كان معاك هو خويا.', 'الراجل لي كيهضر معاك هو خويا.'],
        ['فاش كنكون فالدار، كنقرا الكتوب.'],
        ['ما عرفتش شنو غادي ندير.'],
        ['كيداير الحال اليوم؟'],
        ['ضربني البرد.'],
        ['هاد الماكلة بنينة بزاف!'],
        ['كاين بزاف ديال الناس ف السوق.'],
        ['شفت طوموبيل ف الطريق.', 'شفت راجل ف الطريق.', 'شفت كلب ف الطريق.', 'شفت شجرة ف الطريق.'],
        ['عطيني الخبز.', 'عطيني الماء.', 'عطيني فلوس.', 'عطيني ستيلو.'],
        ['عندي ثلاثة خوت.', 'عندي ربعة خوت.', 'عندي واحد خوت.'],
        ['شريت واحد كيلو ديال التفاح.', 'شريت جوج كيلو ديال التفاح.'],
        ['البنت زوينة بزاف.', 'البنت صغيرة بزاف.', 'البنت كبيرة بزاف.']
        ,['واش نتا مكلخ؟', 'واش نتا مغربي؟', 'واش نتا هنا؟']
        ,['فين كنتي البارح؟', 'فين مشيتي البارح؟']
        ,['ما عنديش فلوس.', 'ما عنديش الوقت.']
        ,['هذا الكتاب ديالك؟', 'هذا القلم ديالك؟']
        ,['شنو كليتي؟', 'واش كليتي؟']
        ,['غادي نمشي نسافر غدا.', 'غادي نمشي نخدم غدا.']
        ,['كنت مريض البارح.', 'كنت خدام البارح.']
        ,['عفاك، عاوني شوية.', 'عفاك، عطيني شوية.']
        ,['ما فهمتش شنو قلتي.', 'ما فهمتش داكشي قلتي.']
        ,['شحال ف الساعة؟', 'شحال دابا الساعة؟']
        ,['بغيت نشرب قهوة.', 'بغيت ناكل قهوة.']
        ,['الدار ديالي نقية.', 'الدار عامرة نقية.']
        ,['فين خدام؟', 'واش خدام؟']
        ,['اليوم هو عيد ميلادي.', 'اليوم ديال عيد ميلادي.']
        ,['كنحس براسي عيان.', 'كنحس بشويا عيان.']
        ,['التليفون ديالي طافي.', 'التليفون ديالي خسر.']
        ,['فين كتسكن تسكن؟', 'فين ساكن تسكن؟']
        ,['واش كلشي مزيان؟', 'واش نتا مزيان؟']
        ,['شنو خاصك؟', 'واش خاصك؟']
        ,['هادشي بزاف صعيب.', 'هادشي شوية صعيب.']
        ,['أنا كنحاول نتعلم الدارجة.', 'أنا باغي نتعلم الدارجة.']
        ,['الطقس ديال زوين اليوم.', 'الطقس اليوم زوين اليوم.']
        ,['عندي ليك سؤال.', 'عندي واحد سؤال.']
        ,['ما كنعرفش هادشي ندير.', 'ما كنعرفش كيفاش ندير.']
        ,['بغيت نمشي نسافر.', 'بغيت نجي نسافر.']
        ,['شحال عام عندك؟', 'من عام عندك؟']
        ,['أنا جاي من المغرب.', 'أنا من من المغرب.']
        ,['هاد الكتاب زوين بزاف.', 'هاد الكتاب غالي بزاف.']
        ,['أش تدير دابا؟', 'شنو تدير دابا؟']
    ]
}
df = pd.DataFrame(data)

In [ ]:
import pandas as pd

data = {
    'darija_masked': [
        "أنا <mask> الكتاب البارح ف القهوة حيت كنت كنتسنى صاحبي.",
        "هو <mask> ف الدار ديال صاحبتو حيت قالوا غادي يتفرجوا ف شي فيلم جديد.",
        "بغيت <mask> د الماء حيت جاني العطش بزاف من بعد ما كليت الحار.",
        "ال<mask> اللي شريت البارح من السوق زوين بزاف و عجب كاع الناس.",
        "<mask> سميتك أ صاحبي؟ شحال هادي ما تلاقينا و نسيت سميتك.",
        "مشيت <mask> للسوق مع ختي الصغيرة باش نشريو شي حوايج للعيد.",
        "غادي <mask> للمدرسة واخا مريض شوية حيت عندي امتحان مهم اليوم.",
        "كنت <mask> مع صحابي ف القهوة حتا جا واحد السيد و بدا كيغوت.",
        "كتبت رسالة <mask> الصديق ديالي اللي ساكن ف فرنسا باش نسول فيه.",
        "ما ت<mask> فهاد الفيلم اللي كيدوز دابا ف التلفازة حيت ما عجبتنيش القصة ديالو.",
        "الراجل لي <mask> معاك البارح ف الحفلة هو خويا الكبير اللي خدام طبيب.",
        "فاش كنكون فالدار بوحدي، كنقرا <mask> ولا كنتفرج ف شي حاجة .",
        "ما عرفتش <mask> غادي ندير فهاد المشكل، حيت جاني صعيب بزاف.",
        "<mask> الحال اليوم؟ واش كاينا شي شتا ولا غير الشمش؟",
        "<mask> البرد بزاف هاد الأيام، ما بقيتش كنقدر نخرج من الدار.",
        "هاد الماكلة اللي طيبتي <mask> بزاف، تبارك الله عليك أ للا.",
        "كاين <mask> ديال الناس ف السوق كيتسناو غير فوقاش تحل الحوانت.",
        "شفت <mask> ف الطريق و هو جاي عندي و بقيت كنتسناه حتى وصل.",
        "عطيني <mask> اللي طلبت منك قبيلة، راني محتاج ليه بزاف دابا.",
        "عندي <mask> خوت، جوج دراري و بنت وحدة، و كلهم كبر مني.",
        "شريت <mask> كيلو ديال التفاح من عند الخضار اللي حدا الجامع.",
        "البنت اللي لابسة الكسوة الحمرا <mask> بزاف و كلشي كيشوف فيها.",
        "واش نتا <mask> بصح؟ ما كنتيقش هادشي اللي كتقول ليا دابا.",
        "فين <mask> البارح ف الليل؟ بقيت كنتسناك و ما جيتيش.",
        "ما عنديش <mask> باش نشري هاداك التلفون، حيت غالي عليا بزاف.",
        "هذا <mask> ديالك؟ لقيتو طايح ف الزنقة قدام داركم.",
        "<mask> كليتي ف الغدا؟ واش كليتي شي حاجة مزيانة ولا والو؟",
        "غادي نمشي <mask> غدا ان شاء الله، إلا ما كانتش الشتا.",
        "كنت <mask> البارح مع العائلة ديالي ف واحد العرس.",
        "عفاك، <mask> شوية ديال السكر ف القهوة ديالي، ما كنبغيش نشربها مسوسة.",
        "ما فهمتش <mask> قلتي ليا، واش ممكن تعاودها بشوية عليا؟",
        "شحال <mask> الساعة دابا؟ واش قرب وقت المغرب ولا مازال؟",
        "بغيت <mask> شي كاس ديال أتاي منعنع، إلى كان ممكن.",
        "الدار اللي حدا <mask> عامرة بالصداع ديما، ما كنقدروش نرتاحو.",
        "<mask> خدام نتا؟ واش عندك شي خدمة مزيانة ولا باقي كتقلب؟",
        "اليوم <mask> عيد ميلادي، و غادي نديرو حفلة صغيرة ف الدار.",
        "كنحس <mask> عيان بزاف هاد الأيام، ما عرفتش واش من الخدمة ولا من البرد.",
        "التليفون ديالي <mask> ليا ف الطاكسي، و ما عرفتش كيفاش غادي نرجعو.",
        "فين <mask> تسكن بالضبط؟ بغيت نجي عندك شي نهار نشوفك.",
        "واش <mask> مزيان دابا؟ سمعت بلي كنتي مريض شوية.",
        "<mask> خاصك باش نكملو هاد الخدمة؟ واش كلشي عندك؟",
        "هادشي اللي وقع <mask> صعيب بزاف، و ما عرفتش شنو غادي نديرو.",
		"أنا <mask> نتعلم الدارجة بشوية بشوية، حيت كتعجبني بزاف.",
		"الطقس ديال <mask> زوين بزاف، ما سخون ما بارد.",
		"عندي <mask> سؤال مهم بزاف بغيت نسولك عليه، واش عندك الوقت؟",
		"ما كنعرفش <mask> ندير هادشي بوحدي، واش ممكن تعاوني؟",
		"بغيت <mask> نسافر لشي بلاصة بعيدة، باش نرتاح شوية من هاد الروتين.",
		"<mask> عام عندك أ صاحبي؟ نسيت شحال قلتي ليا.",
		"أنا <mask> من المغرب، و كنفتخر ببلادي بزاف.",
		"هاد الكتاب اللي قريت <mask> بزاف، و أثر فيا.",
		"<mask> كدير دابا؟ واش مشغول ولا نقدر نهضر معاك؟"]
}
df=pd.DataFrame(data)

In [ ]:
df.tail(30)

,darija_masked
21,البنت اللي لابسة الكسوة الحمرا <mask> بزاف و ك...
22,واش نتا <mask> بصح؟ ما كنتيقش هادشي اللي كتقول...
23,فين <mask> البارح ف الليل؟ بقيت كنتسناك و ما ج...
24,ما عنديش <mask> باش نشري هاداك التلفون، حيت غا...
25,هذا <mask> ديالك؟ لقيتو طايح ف الزنقة قدام داركم.
26,<mask> كليتي ف الغدا؟ واش كليتي شي حاجة مزيانة...
27,غادي نمشي <mask> غدا ان شاء الله، إلا ما كانتش...
28,كنت <mask> البارح مع العائلة ديالي ف واحد العرس.
29,عفاك، <mask> شوية ديال السكر ف القهوة ديالي، م...
30,ما فهمتش <mask> قلتي ليا، واش ممكن تعاودها بشو...


In [ ]:
from datasets import Dataset

In [ ]:
ds=Dataset.from_pandas(df)
ds

Dataset({
    features: ['darija_masked'],
    num_rows: 51
})

# Load Models

In [ ]:
from transformers import pipeline
import torch
from google.colab import userdata

In [ ]:
token=userdata.get("hf_read")

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
MODEL_LIST=[
  "BounharAbdelaziz/XLM-RoBERTa-Morocco",
  "google-bert/bert-base-multilingual-cased",
  "FacebookAI/xlm-roberta-large",
  "aubmindlab/bert-base-arabertv02",
  "SI2M-Lab/DarijaBERT",
  "BounharAbdelaziz/ModernBERT-Morocco"
]
MODEL_LIST

['BounharAbdelaziz/XLM-RoBERTa-Morocco',
 'google-bert/bert-base-multilingual-cased',
 'FacebookAI/xlm-roberta-large',
 'aubmindlab/bert-base-arabertv02',
 'SI2M-Lab/DarijaBERT']

In [ ]:
# load models function
def load_model(model_name,device):
    pipe=pipeline(task="fill-mask",model=model_name,device=device,token=token)
    return pipe

In [ ]:
# fill mask
from tqdm import tqdm
target_name="<mask>"
for model_name in tqdm(MODEL_LIST):
  model=load_model(model_name,device)
  name=model.tokenizer.all_special_tokens[-1]
  ds=ds.map(lambda x:{"darija_masked":x.replace(target_name,name)},input_columns="darija_masked")# replace
  target_name=name
  ds=ds.map(lambda x: {model_name:model(x)},input_columns="darija_masked",batched=True,batch_size=8)
  ds=ds.map(lambda x: {model_name:x[0]["sequence"]},input_columns=model_name)
  del model

  0%|          | 0/5 [00:00<?, ?it/s]Device set to use cpu


Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

 20%|██        | 1/5 [01:00<04:01, 60.38s/it]Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


...

 80%|████████  | 4/5 [02:58<00:40, 40.68s/it]The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cpu


Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

100%|██████████| 5/5 [03:15<00:00, 39.11s/it]


In [ ]:
df_result=ds.to_pandas()
df_result

,darija_masked,BounharAbdelaziz/XLM-RoBERTa-Morocco,google-bert/bert-base-multilingual-cased,FacebookAI/xlm-roberta-large,aubmindlab/bert-base-arabertv02,SI2M-Lab/DarijaBERT
0,أنا [MASK] الكتاب البارح ف القهوة حيت كنت كنتس...,أنا سمعت الكتاب البارح ف القهوة حيت كنت كنتسنى...,أنا في الكتاب البارح ف القهوة حيت كنت كنتسنى ص...,أنا كنت الكتاب البارح ف القهوة حيت كنت كنتسنى ...,أنا قري الكتاب البارح ف القهوة حيت كنت كنتسنى ...,انا قريت الكتاب البارح ف القهوة حيت كنت كنتسنى...
1,هو [MASK] ف الدار ديال صاحبتو حيت قالوا غادي ي...,هو كان ف الدار ديال صاحبتو حيت قالوا غادي يتفر...,هوي ف الدار ديال صاحبتو حيت قالوا غادي يتفرجوا...,هو كان ف الدار ديال صاحبتو حيت قالوا غادي يتفر...,هو كان ف الدار ديال صاحبتو حيت قالوا غادي يتفر...,هو كايتفرج ف الدار ديال صاحبتو حيت قالوا غادي ...
2,بغيت [MASK] د الماء حيت جاني العطش بزاف من بعد...,بغيت كأس د الماء حيت جاني العطش بزاف من بعد ما...,بغيت من د الماء حيت جاني العطش بزاف من بعد ما ...,بغيت ا د الماء حيت جاني العطش بزاف من بعد ما ك...,بغيت كلي د الماء حيت جاني العطش بزاف من بعد ما...,بغيت هديل د الماء حيت جاني العطش بزاف من بعد م...
3,ال[MASK] اللي شريت البارح من السوق زوين بزاف و...,الورد اللي شريت البارح من السوق زوين بزاف و عج...,اللي اللي شريت البارح من السوق زوين بزاف و عجب...,الطبق اللي شريت البارح من السوق زوين بزاف و عج...,ال 5 اللي شريت البارح من السوق زوين بزاف و عجب...,الvi اللي شريت البارح من السوق زوين بزاف و عجب...
4,[MASK] سميتك أ صاحبي؟ شحال هادي ما تلاقينا و ن...,شنو سميتك أ صاحبي؟ شحال هادي ما تلاقينا و نسيت...,يا سميتك أ صاحبي ؟ شحال هادي ما تلاقينا و نسيت...,ما سميتك أ صاحبي؟ شحال هادي ما تلاقينا و نسيت ...,نسيت سميتك أ صاحبي ؟ شحال هادي ما تلاقينا و نس...,""" سميتك ا صاحبي ؟ شحال هادي ما تلاقينا و نسيت ..."
5,مشيت [MASK] للسوق مع ختي الصغيرة باش نشريو شي ...,مشيت اليوم للسوق مع ختي الصغيرة باش نشريو شي ح...,مشيت : للسوق مع ختي الصغيرة باش نشريو شي حوايج...,مشيت اليوم للسوق مع ختي الصغيرة باش نشريو شي ح...,مشيت انا للسوق مع ختي الصغيرة باش نشريو شي حوا...,مشيت مشيت للسوق مع ختي الصغيرة باش نشريو شي حو...
6,غادي [MASK] للمدرسة واخا مريض شوية حيت عندي ام...,غادي اليوم للمدرسة واخا مريض شوية حيت عندي امت...,غاديم للمدرسة واخا مريض شوية حيت عندي امتحان م...,غادي خرج للمدرسة واخا مريض شوية حيت عندي امتحا...,غادي ندخل للمدرسة واخا مريض شوية حيت عندي امتح...,غادي نمشي للمدرسة واخا مريض شوية حيت عندي امتح...
7,كنت [MASK] مع صحابي ف القهوة حتا جا واحد السيد...,كنت غير مع صحابي ف القهوة حتا جا واحد السيد و ...,كنت أيضا مع صحابي ف القهوة حتا جا واحد السيد و...,كنت انا مع صحابي ف القهوة حتا جا واحد السيد و ...,كنت قاعد مع صحابي ف القهوة حتا جا واحد السيد و...,كنت جالس مع صحابي ف القهوة حتا جا واحد السيد و...
8,كتبت رسالة [MASK] الصديق ديالي اللي ساكن ف فرن...,كتبت رسالة ل الصديق ديالي اللي ساكن ف فرنسا با...,كتبت رسالة إلى الصديق ديالي اللي ساكن ف فرنسا ...,كتبت رسالة الى الصديق ديالي اللي ساكن ف فرنسا ...,كتبت رسالة إلى الصديق ديالي اللي ساكن ف فرنسا ...,كتبت رسالة ل الصديق ديالي اللي ساكن ف فرنسا با...
9,ما ت[MASK] فهاد الفيلم اللي كيدوز دابا ف التلف...,ما تفرج فهاد الفيلم اللي كيدوز دابا ف التلفازة...,ما تت فهاد الفيلم اللي كيدوز دابا ف التلفازة ح...,ما تدري فهاد الفيلم اللي كيدوز دابا ف التلفازة...,ما تش فهاد الفيلم اللي كيدوز دابا ف التلفازة ح...,ما ت تفرجو فهاد الفيلم اللي كيدوز دابا ف التلف...


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_result)

https://docs.google.com/spreadsheets/d/1HCW3zsTlJquxuk1Uf_dJH48RMXJ4h1H4pouApmHcZ3A#gid=0


# Use Masked Fill As Text Gen

In [ ]:
from transformers import AutoModelForMaskedLM,AutoTokenizer

In [ ]:
# load model and tokenizer
model=AutoModelForMaskedLM.from_pretrained("FacebookAI/xlm-roberta-large",token=token).to("cuda")
tokenizer=AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-large",token=token)

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at FacebookAI/xlm-roberta-large were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
# create pipe
from transformers import pipeline
pipe=pipeline(task="fill-mask",model=model,tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
#response="ﺣﺘﻲ ﺣﻠﻴﻤﺔ ﺍﻧﺖ ﺇﻧﺴﺎﻧﺔ ﺧﻠﻮﻗﺔ ﻭﺣﻨﺎ ﻓﻲ ﺇﻧﺘﻈﺎﺭﻙ ﻓﺸﻬﺮ"
#response="ﺍﻟﺒﻨﺎﺕ ﺩﻋﻴﻮ ﻣﻌﺎﻳﺎ ﻧﺪﻳﺮ ﺩﻭﻳﺮﺗﻲ ﺭﺍﻩ ﺩﻋﻮﺓ"
response="الدكاء الإصطناعي هو"
for i in range(10):
  response+="<mask>"
  response=pipe(response)[0]["sequence"]
  print(response)

الدكاء الإصطناعي هو:
الدكاء الإصطناعي هو:
الدكاء الإصطناعي هو:
الدكاء الإصطناعي هو:
الدكاء الإصطناعي هو:
الدكاء الإصطناعي هو:
الدكاء الإصطناعي هو:
الدكاء الإصطناعي هو:
الدكاء الإصطناعي هو:
الدكاء الإصطناعي هو:


# Fill MAsk: Embedding Bench Dataset

In [ ]:
# prepare another data
from datasets import load_dataset
ds=load_dataset("atlasia/Morocco-Darija-Sentence-Embedding-Benchmark",split="test",token=token)
ds

README.md:   0%|          | 0.00/3.05k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/33.1k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/725 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence1', 'sentence2', 'score'],
    num_rows: 725
})

In [ ]:
# extract long sentences
long_sentence=ds.filter(lambda x: len(x)>90,input_columns="sentence1")
long_sentence=long_sentence.select_columns("sentence1")

In [ ]:
# create masked sentences
import random
def mask_sen(sentence):
  tokens=sentence.split(" ")
  l=len(tokens)
  idx=random.randint(0,l-1)
  tokens[idx]="<mask>"
  return " ".join(tokens)
long_sentence=long_sentence.map(lambda x: {"masked_sentece":mask_sen(x["sentence1"])})
long_sentence

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence1', 'masked_sentece'],
    num_rows: 62
})

In [ ]:
long_sentence.to_pandas()

,sentence1,masked_sentece
0,كنفكر نشري شي طوموبيل جديدة ولكن مازال ما قررت...,كنفكر نشري شي طوموبيل جديدة ولكن مازال ما قررت...
1,ما كنعرفش نطيب الكسكس بالخضرة مزيان بحال الوال...,ما <mask> نطيب الكسكس بالخضرة مزيان بحال الوال...
2,كنقرا واحد الرواية ديال واحد الكاتب مغربي مشهو...,كنقرا واحد الرواية ديال واحد الكاتب <mask> مشه...
3,اليوم فالخدمة، كان عندي اجتماع مهم بزاف مع الم...,اليوم فالخدمة، كان عندي اجتماع مهم <mask> مع ا...
4,مشيت لسوق الخميس اللي فات وشريت زربية فاسية زو...,مشيت لسوق الخميس اللي فات وشريت زربية فاسية زو...
...,...,...
57,كنقرا الجرائد المغربية كل نهار باش نبقى على اط...,كنقرا الجرائد المغربية كل نهار باش <mask> على ...
58,كنبغي نتفرج فالماتشات ديال المنتخب الوطني المغ...,كنبغي نتفرج فالماتشات ديال المنتخب <mask> المغ...
59,خاصني نصاوب جواز السفر ديالي حيت بغيت نسافر، و...,خاصني نصاوب جواز السفر ديالي حيت بغيت <mask> و...
60,كنفكر ندير شي مشروع صغير ديالي، ولكن مازال ما ...,كنفكر ندير شي مشروع صغير ديالي، ولكن <mask> ما...


In [ ]:
from tqdm import tqdm
import gc
target_name="<mask>"
for model_name in tqdm(MODEL_LIST):
  model=load_model(model_name,"cuda")
  name=model.tokenizer.all_special_tokens[-1]
  long_sentence=long_sentence.map(lambda x:{"masked_sentece":x.replace(target_name,name)},input_columns="masked_sentece")# replace
  target_name=name
  long_sentence=long_sentence.map(lambda x: {model_name:model(x)},input_columns="masked_sentece",batched=True,batch_size=8)
  long_sentence=long_sentence.map(lambda x: {model_name:x[0]["sequence"]},input_columns=model_name)
  gc.collect()
  torch.cuda.empty_cache()
  del model

  0%|          | 0/5 [00:00<?, ?it/s]Device set to use cuda


Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

 20%|██        | 1/5 [00:19<01:18, 19.62s/it]

...

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cuda


Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

100%|██████████| 5/5 [01:35<00:00, 19.19s/it]


In [ ]:
result_df=long_sentence.to_pandas()
result_df.head()

,sentence1,masked_sentece,BounharAbdelaziz/XLM-RoBERTa-Morocco,google-bert/bert-base-multilingual-cased,FacebookAI/xlm-roberta-large,aubmindlab/bert-base-arabertv02,SI2M-Lab/DarijaBERT
0,كنفكر نشري شي طوموبيل جديدة ولكن مازال ما قررت...,كنفكر نشري شي طوموبيل جديدة ولكن مازال ما قررت...,كنفكر نشري شي طوموبيل جديدة ولكن مازال ما قررت...,كنفكر نشري شي طوموبيل جديدة ولكن مازال ما قررت...,كنفكر نشري شي طوموبيل جديدة ولكن مازال ما قررت...,كنفكر نشري شي طوموبيل جديدة ولكن مازال ما قررت...,كنفكر نشري شي طوموبيل جديدة ولكن مازال ما قررت...
1,ما كنعرفش نطيب الكسكس بالخضرة مزيان بحال الوال...,ما [MASK] نطيب الكسكس بالخضرة مزيان بحال الوال...,ما قادر نطيب الكسكس بالخضرة مزيان بحال الوالدة...,مالا نطيب الكسكس بالخضرة مزيان بحال الوالدة دي...,ما كنا نطيب الكسكس بالخضرة مزيان بحال الوالدة ...,مادي نطيب الكسكس بالخضرة مزيان بحال الوالدة دي...,ما قدرتش نطيب الكسكس بالخضرة مزيان بحال الوالد...
2,كنقرا واحد الرواية ديال واحد الكاتب مغربي مشهو...,كنقرا واحد الرواية ديال واحد الكاتب [MASK] مشه...,كنقرا واحد الرواية ديال واحد الكاتب عربي مشهور...,كنقرا واحد الرواية ديال واحد الكاتبة مشهور كيه...,كنقرا واحد الرواية ديال واحد الكاتب المغربي مش...,كنقرا واحد الرواية ديال واحد الكاتب اللي مشهور...,كنقرا واحد الرواية ديال واحد الكاتب شعبي مشهور...
3,اليوم فالخدمة، كان عندي اجتماع مهم بزاف مع الم...,اليوم فالخدمة، كان عندي اجتماع مهم [MASK] مع ا...,اليوم فالخدمة، كان عندي اجتماع مهم جدا مع المد...,اليوم فالخدمة ، كان عندي اجتماع مهما مع المدير...,اليوم فالخدمة، كان عندي اجتماع مهم جدا مع المد...,اليوم فالخدمة ، كان عندي اجتماع مهم جدا مع الم...,اليوم فالخدمة ، كان عندي اجتماع مهم ليوما مع ا...
4,مشيت لسوق الخميس اللي فات وشريت زربية فاسية زو...,مشيت لسوق الخميس اللي فات وشريت زربية فاسية زو...,مشيت لسوق الخميس اللي فات وشريت زربية فاسية زو...,مشيت لسوق الخميس اللي فات وشريت زربية فاسية زو...,مشيت لسوق الخميس اللي فات وشريت زربية فاسية زو...,مشيت لسوق الخميس اللي فات وشريت زربية فاسية زو...,مشيت لسوق الخميس اللي فات وشريت زربية فاسية زو...


# Tarjma Bench

In [ ]:
from datasets import load_dataset
tarjama_ds=load_dataset("atlasia/TerjamaBench",split="test",token=token)
tarjama_ds

test-00000-of-00001.parquet:   0%|          | 0.00/67.1k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/850 [00:00<?, ? examples/s]

Dataset({
    features: ['topic', 'subtopic', 'Arabizi', 'English', 'Darija', 'annotator_dialect'],
    num_rows: 850
})

In [ ]:
set(tarjama_ds["topic"])

{'common_phrases',
 'dialect_variation',
 'educational',
 'humor',
 'idioms',
 'incorrect_spellings',
 'long_sentences',
 'mixed_language',
 'named_entities',
 'numeric_and_date',
 'religion',
 'single_words'}

In [ ]:
selected_topic=["religion","idioms","long_sentences","common_phrases","religion","humor"]
length=[15,10,25,10,10]

In [ ]:
sentences=[]
topics=[]
for topic,l in zip(selected_topic,length):
  filter_ds=tarjama_ds.filter(lambda x: x["topic"]==topic and len(x["Darija"])>20)
  sentences+=filter_ds["Darija"][:l]
  topics+=filter_ds["topic"][:l]
sentences

In [ ]:
import random
mask_sentences=[]
for sentence in sentences:
  tokens=sentence.split(" ")
  idx=random.randint(0,len(tokens)-1)
  tokens[idx]="<mask>"
  mask_sentences.append(" ".join(tokens))
#mask_sentences

In [ ]:
import pandas as pd
df=pd.DataFrame({"sentence":sentences,"mask_sentence":mask_sentences,"topic":topics})
df.head()

,sentence,mask_sentence,topic
0,لا حول و لا قوة الا بالله,لا حول و لا قوة <mask> بالله,religion
1,الله يسمح لينا كاملين,الله <mask> لينا كاملين,religion
2,غير صبر، ربي غايفاجيها,غير صبر، ربي <mask>,religion
3,الله يسمحلينا من الوالدين,الله يسمحلينا <mask> الوالدين,religion
4,لا اله الا الله محمد رسول الله,لا اله الا الله محمد <mask> الله,religion


In [ ]:
# generator
from datasets import Dataset
ds=Dataset.from_pandas(df)
ds

Dataset({
    features: ['sentence', 'mask_sentence', 'topic'],
    num_rows: 70
})

In [ ]:
from tqdm import tqdm
import gc
target_name="<mask>"
for model_name in tqdm(MODEL_LIST):
  model=load_model(model_name,"cuda")
  name=model.tokenizer.all_special_tokens[-1]
  ds=ds.map(lambda x:{"mask_sentence":x.replace(target_name,name)},input_columns="mask_sentence")# replace
  target_name=name
  ds=ds.map(lambda x: {model_name:model(x)},input_columns="mask_sentence",batched=True,batch_size=8)
  ds=ds.map(lambda x: {model_name:x[0]["sequence"]},input_columns=model_name)
  gc.collect()
  torch.cuda.empty_cache()
  del model

  0%|          | 0/6 [00:00<?, ?it/s]Device set to use cuda


Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

 17%|█▋        | 1/6 [00:23<01:56, 23.27s/it]Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda


...

Device set to use cuda


Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

100%|██████████| 6/6 [01:34<00:00, 15.77s/it]


In [ ]:
df=ds.to_pandas()
df

,sentence,mask_sentence,topic,BounharAbdelaziz/XLM-RoBERTa-Morocco,google-bert/bert-base-multilingual-cased,FacebookAI/xlm-roberta-large,aubmindlab/bert-base-arabertv02,SI2M-Lab/DarijaBERT,BounharAbdelaziz/ModernBERT-Morocco
0,لا حول و لا قوة الا بالله,لا حول و لا قوة [MASK] بالله,religion,لا حول و لا قوة الا بالله,لا حول و لا قوة. بالله,لا حول و لا قوة إلا بالله,لا حول و لا قوة إلا بالله,لا حول و لا قوة با بالله,لا حول و لا قوة الا بالله
1,الله يسمح لينا كاملين,الله [MASK] لينا كاملين,religion,الله يسمح لينا كاملين,الله - لينا كاملين,الله يجعل لينا كاملين,الله يحفظ لينا كاملين,الله يصوب لينا كاملين,الله يسمح لينا كاملين
2,غير صبر، ربي غايفاجيها,غير صبر، ربي [MASK],religion,غير صبر، ربي.,غير صبر ، ربي.,غير صبر، ربي,غير صبر ، ربي.,غير صبر ، ربي سبحانو,غير صبر، ربي.
3,الله يسمحلينا من الوالدين,الله يسمحلينا [MASK] الوالدين,religion,الله يسمحلينا من الوالدين,الله يسمحلينا إلى الوالدين,الله يسمحلينا مع الوالدين,الله يسمحلينا ببر الوالدين,الله يسمحلينا منهوم الوالدين,الله يسمحلينا على الوالدين
4,لا اله الا الله محمد رسول الله,لا اله الا الله محمد [MASK] الله,religion,لا اله الا الله محمد رسول الله,لا اله الا الله محمد رسول الله,لا اله الا الله محمد رسول الله,لا اله الا الله محمد رسول الله,لا اله الا الله محمد وال الله,لا اله الا الله محمد و الله
...,...,...,...,...,...,...,...,...,...
65,إنا لله وإنا إليه راجعون,إنا لله [MASK] إليه راجعون,religion,إنا لله وإن إليه راجعون,إنا للهم إليه راجعون,إنا لله وإن إليه راجعون,إنا لله وإنا إليه راجعون,انا لله وان اليه راجعون,إنا لله وإنا إليه راجعون
66,الجفاف أخويا، الله يرحمنا وصافي,الجفاف أخويا، الله يرحمنا [MASK],religion,الجفاف أخويا، الله يرحمنا.,الجفاف أخويا ، الله يرحمنا ،,الجفاف أخويا، الله يرحمنا جميعا,الجفاف أخويا ، الله يرحمنا.,الجفاف اخويا ، الله يرحمنا جميعا,الجفاف أخويا، الله يرحمنا.
67,هادشي اللي كيديرو فيه هاد الناس حرام، واش ماشي...,هادشي اللي كيديرو [MASK] هاد الناس حرام، واش م...,religion,هادشي اللي كيديرو فيه هاد الناس حرام، واش ماشي...,هادشي اللي كيديرو ، هاد الناس حرام ، واش ماشي ...,هادشي اللي كيديرو على هاد الناس حرام، واش ماشي...,هادشي اللي كيديرو مع هاد الناس حرام ، واش ماشي...,هادشي اللي كيديروالوش هاد الناس حرام ، واش ماش...,هادشي اللي كيديرو فيه هاد الناس حرام، واش ماشي...
68,هادوا راه ما كيخافوش من الله,هادوا راه ما [MASK] من الله,religion,هادوا راه ماشي من الله,هادوا راه ما إن من الله,هادوا راه مابين من الله,هادوا راه ما نخاف من الله,هادوا راه ما رايت من الله,هادوا راه ما حسن من الله


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1J1O_Uk0lYAJDe3FnAGNlqRo6wDrZOl6o1pHV5EQ62ko#gid=0
